# Project Proposal (Matt)

The ETL Project involved performing an Extract-Transform-Load (ETL) process on airline flight performance data and customer tweet data from February 2015. The members of the group are Sedra Kurdi, Myles Bridges, Natalie Myers, and Matthew Kennedy. The datasets chosen were found at https://www.kaggle.com/usdot/flight-delays and https://www.kaggle.com/crowdflower/twitter-airline-sentiment, and were in CSV and SQLite format respectively. The approach was to use the SQLAlchemy and Pandas modules in Python to extract and transform the data. SQLAlchemy was then used to load the transformed data into a PostgreSQL database.

# Project Code

In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func
from sqlalchemy import Table, Column, Integer, String, MetaData

## Data Extraction

### Flight Data (Myles & Sedra)

In [2]:
#ADD CODE HERE

### Tweet Data (Natalie)

In [3]:
sqlite_engine = create_engine("sqlite:///Tweet_Data/database.sqlite")

In [4]:
inspector = inspect(sqlite_engine)
inspector.get_table_names()

['Tweets']

In [5]:
data_tweets = pd.read_sql_table('Tweets',sqlite_engine)

data_tweets

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,567588278875213824,neutral,1.0000,,,Delta,,JetBlueNews,,0,@JetBlue's new CEO seeks the right balance to ...,,2015-02-16 23:36:05 -0800,USA,Sydney
1,567590027375702016,negative,1.0000,Can't Tell,0.6503,Delta,,nesi_1992,,0,@JetBlue is REALLY getting on my nerves !! 😡😡 ...,,2015-02-16 23:43:02 -0800,undecided,Pacific Time (US & Canada)
2,567591480085463040,negative,1.0000,Late Flight,0.346,United,,CPoutloud,,0,@united yes. We waited in line for almost an h...,,2015-02-16 23:48:48 -0800,"Washington, DC",
3,567592368451248130,negative,1.0000,Late Flight,1,United,,brenduch,,0,@united the we got into the gate at IAH on tim...,,2015-02-16 23:52:20 -0800,,Buenos Aires
4,567594449874587648,negative,1.0000,Customer Service Issue,0.3451,Southwest,,VahidESQ,,0,@SouthwestAir its cool that my bags take a bit...,,2015-02-17 00:00:36 -0800,"Los Angeles, CA",Pacific Time (US & Canada)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14480,570309308937842688,neutral,0.6869,,,Delta,,Oneladyyouadore,,0,@JetBlue I hope so because I fly very often an...,,2015-02-24 11:48:29 -0800,Georgia,Quito
14481,570309340952993796,neutral,1.0000,,,US Airways,,DebbiMcGinnis,,0,@USAirways is a DM possible if you aren't foll...,,2015-02-24 11:48:37 -0800,Missourah,Hawaii
14482,570309345281486848,positive,0.6469,,,Delta,,jaxbra,,0,@JetBlue Yesterday on my way from EWR to FLL j...,,2015-02-24 11:48:38 -0800,"east brunswick, nj",Atlantic Time (Canada)
14483,570310144459972608,negative,1.0000,Customer Service Issue,1,US Airways,,GAKotsch,,0,@USAirways and when will one of these agents b...,,2015-02-24 11:51:48 -0800,,Atlantic Time (Canada)


## Data Transformation

### Flight Data (Myles & Sedra)

In [6]:
#ADD CODE HERE

### Tweet Data (Natalie)

In [7]:
data_tweets = data_tweets.drop(['negativereason', 'negativereason_confidence', 'airline_sentiment_gold', 
                                'negativereason_gold', 
                                'tweet_coord', 'tweet_location', 'user_timezone'],axis=1)
data_tweets = data_tweets.dropna()
data_tweets.sort_values("tweet_created")
data_tweets["tweet_id"] = data_tweets["tweet_id"].astype(str)
data_tweets

,tweet_id,airline_sentiment,airline_sentiment_confidence,airline,name,retweet_count,text,tweet_created
0,567588278875213824,neutral,1.0000,Delta,JetBlueNews,0,@JetBlue's new CEO seeks the right balance to ...,2015-02-16 23:36:05 -0800
1,567590027375702016,negative,1.0000,Delta,nesi_1992,0,@JetBlue is REALLY getting on my nerves !! 😡😡 ...,2015-02-16 23:43:02 -0800
2,567591480085463040,negative,1.0000,United,CPoutloud,0,@united yes. We waited in line for almost an h...,2015-02-16 23:48:48 -0800
3,567592368451248130,negative,1.0000,United,brenduch,0,@united the we got into the gate at IAH on tim...,2015-02-16 23:52:20 -0800
4,567594449874587648,negative,1.0000,Southwest,VahidESQ,0,@SouthwestAir its cool that my bags take a bit...,2015-02-17 00:00:36 -0800
...,...,...,...,...,...,...,...,...
14480,570309308937842688,neutral,0.6869,Delta,Oneladyyouadore,0,@JetBlue I hope so because I fly very often an...,2015-02-24 11:48:29 -0800
14481,570309340952993796,neutral,1.0000,US Airways,DebbiMcGinnis,0,@USAirways is a DM possible if you aren't foll...,2015-02-24 11:48:37 -0800
14482,570309345281486848,positive,0.6469,Delta,jaxbra,0,@JetBlue Yesterday on my way from EWR to FLL j...,2015-02-24 11:48:38 -0800
14483,570310144459972608,negative,1.0000,US Airways,GAKotsch,0,@USAirways and when will one of these agents b...,2015-02-24 11:51:48 -0800


In [8]:
data_tweets['airline'] = data_tweets["airline"].replace("United","UA")
data_tweets['airline'] = data_tweets["airline"].replace("Delta","DL")
data_tweets['airline'] = data_tweets["airline"].replace("Southwest","WN")
data_tweets['airline'] = data_tweets["airline"].replace("American","AA")
data_tweets['airline'] = data_tweets["airline"].replace("US Airways","US")
data_tweets['airline'] = data_tweets["airline"].replace("Virgin America","VX")
data_tweets

,tweet_id,airline_sentiment,airline_sentiment_confidence,airline,name,retweet_count,text,tweet_created
0,567588278875213824,neutral,1.0000,DL,JetBlueNews,0,@JetBlue's new CEO seeks the right balance to ...,2015-02-16 23:36:05 -0800
1,567590027375702016,negative,1.0000,DL,nesi_1992,0,@JetBlue is REALLY getting on my nerves !! 😡😡 ...,2015-02-16 23:43:02 -0800
2,567591480085463040,negative,1.0000,UA,CPoutloud,0,@united yes. We waited in line for almost an h...,2015-02-16 23:48:48 -0800
3,567592368451248130,negative,1.0000,UA,brenduch,0,@united the we got into the gate at IAH on tim...,2015-02-16 23:52:20 -0800
4,567594449874587648,negative,1.0000,WN,VahidESQ,0,@SouthwestAir its cool that my bags take a bit...,2015-02-17 00:00:36 -0800
...,...,...,...,...,...,...,...,...
14480,570309308937842688,neutral,0.6869,DL,Oneladyyouadore,0,@JetBlue I hope so because I fly very often an...,2015-02-24 11:48:29 -0800
14481,570309340952993796,neutral,1.0000,US,DebbiMcGinnis,0,@USAirways is a DM possible if you aren't foll...,2015-02-24 11:48:37 -0800
14482,570309345281486848,positive,0.6469,DL,jaxbra,0,@JetBlue Yesterday on my way from EWR to FLL j...,2015-02-24 11:48:38 -0800
14483,570310144459972608,negative,1.0000,US,GAKotsch,0,@USAirways and when will one of these agents b...,2015-02-24 11:51:48 -0800


In [9]:
data_tweets_new = data_tweets['tweet_created'].str.split(" ",n=2,expand=True)

In [10]:
data_tweets["tweet_date"] = data_tweets_new[0]
data_tweets

,tweet_id,airline_sentiment,airline_sentiment_confidence,airline,name,retweet_count,text,tweet_created,tweet_date
0,567588278875213824,neutral,1.0000,DL,JetBlueNews,0,@JetBlue's new CEO seeks the right balance to ...,2015-02-16 23:36:05 -0800,2015-02-16
1,567590027375702016,negative,1.0000,DL,nesi_1992,0,@JetBlue is REALLY getting on my nerves !! 😡😡 ...,2015-02-16 23:43:02 -0800,2015-02-16
2,567591480085463040,negative,1.0000,UA,CPoutloud,0,@united yes. We waited in line for almost an h...,2015-02-16 23:48:48 -0800,2015-02-16
3,567592368451248130,negative,1.0000,UA,brenduch,0,@united the we got into the gate at IAH on tim...,2015-02-16 23:52:20 -0800,2015-02-16
4,567594449874587648,negative,1.0000,WN,VahidESQ,0,@SouthwestAir its cool that my bags take a bit...,2015-02-17 00:00:36 -0800,2015-02-17
...,...,...,...,...,...,...,...,...,...
14480,570309308937842688,neutral,0.6869,DL,Oneladyyouadore,0,@JetBlue I hope so because I fly very often an...,2015-02-24 11:48:29 -0800,2015-02-24
14481,570309340952993796,neutral,1.0000,US,DebbiMcGinnis,0,@USAirways is a DM possible if you aren't foll...,2015-02-24 11:48:37 -0800,2015-02-24
14482,570309345281486848,positive,0.6469,DL,jaxbra,0,@JetBlue Yesterday on my way from EWR to FLL j...,2015-02-24 11:48:38 -0800,2015-02-24
14483,570310144459972608,negative,1.0000,US,GAKotsch,0,@USAirways and when will one of these agents b...,2015-02-24 11:51:48 -0800,2015-02-24


In [11]:
data_tweets_time= data_tweets_new[1].str.split(":",n=2, expand=True)
data_tweets["tweet_time"]=data_tweets_time[0]  + ":" + data_tweets_time[1]
data_tweets

,tweet_id,airline_sentiment,airline_sentiment_confidence,airline,name,retweet_count,text,tweet_created,tweet_date,tweet_time
0,567588278875213824,neutral,1.0000,DL,JetBlueNews,0,@JetBlue's new CEO seeks the right balance to ...,2015-02-16 23:36:05 -0800,2015-02-16,23:36
1,567590027375702016,negative,1.0000,DL,nesi_1992,0,@JetBlue is REALLY getting on my nerves !! 😡😡 ...,2015-02-16 23:43:02 -0800,2015-02-16,23:43
2,567591480085463040,negative,1.0000,UA,CPoutloud,0,@united yes. We waited in line for almost an h...,2015-02-16 23:48:48 -0800,2015-02-16,23:48
3,567592368451248130,negative,1.0000,UA,brenduch,0,@united the we got into the gate at IAH on tim...,2015-02-16 23:52:20 -0800,2015-02-16,23:52
4,567594449874587648,negative,1.0000,WN,VahidESQ,0,@SouthwestAir its cool that my bags take a bit...,2015-02-17 00:00:36 -0800,2015-02-17,00:00
...,...,...,...,...,...,...,...,...,...,...
14480,570309308937842688,neutral,0.6869,DL,Oneladyyouadore,0,@JetBlue I hope so because I fly very often an...,2015-02-24 11:48:29 -0800,2015-02-24,11:48
14481,570309340952993796,neutral,1.0000,US,DebbiMcGinnis,0,@USAirways is a DM possible if you aren't foll...,2015-02-24 11:48:37 -0800,2015-02-24,11:48
14482,570309345281486848,positive,0.6469,DL,jaxbra,0,@JetBlue Yesterday on my way from EWR to FLL j...,2015-02-24 11:48:38 -0800,2015-02-24,11:48
14483,570310144459972608,negative,1.0000,US,GAKotsch,0,@USAirways and when will one of these agents b...,2015-02-24 11:51:48 -0800,2015-02-24,11:51


In [12]:
data_tweets= data_tweets.drop(["tweet_created"],axis=1)
data_tweets

,tweet_id,airline_sentiment,airline_sentiment_confidence,airline,name,retweet_count,text,tweet_date,tweet_time
0,567588278875213824,neutral,1.0000,DL,JetBlueNews,0,@JetBlue's new CEO seeks the right balance to ...,2015-02-16,23:36
1,567590027375702016,negative,1.0000,DL,nesi_1992,0,@JetBlue is REALLY getting on my nerves !! 😡😡 ...,2015-02-16,23:43
2,567591480085463040,negative,1.0000,UA,CPoutloud,0,@united yes. We waited in line for almost an h...,2015-02-16,23:48
3,567592368451248130,negative,1.0000,UA,brenduch,0,@united the we got into the gate at IAH on tim...,2015-02-16,23:52
4,567594449874587648,negative,1.0000,WN,VahidESQ,0,@SouthwestAir its cool that my bags take a bit...,2015-02-17,00:00
...,...,...,...,...,...,...,...,...,...
14480,570309308937842688,neutral,0.6869,DL,Oneladyyouadore,0,@JetBlue I hope so because I fly very often an...,2015-02-24,11:48
14481,570309340952993796,neutral,1.0000,US,DebbiMcGinnis,0,@USAirways is a DM possible if you aren't foll...,2015-02-24,11:48
14482,570309345281486848,positive,0.6469,DL,jaxbra,0,@JetBlue Yesterday on my way from EWR to FLL j...,2015-02-24,11:48
14483,570310144459972608,negative,1.0000,US,GAKotsch,0,@USAirways and when will one of these agents b...,2015-02-24,11:51


In [13]:
#creating a groupby for airline codes
data_tweets_grouped = data_tweets.groupby(["airline"])

In [14]:
#creating separate dataframes for different airlines
AA_df = data_tweets_grouped.get_group("AA")

In [15]:
#Creating new dataframes based on type of review 

AA_grouped = AA_df.groupby("airline_sentiment")
AA_positive = AA_grouped.get_group("positive")
AA_positive=AA_positive.reset_index(drop=True)

AA_positive

In [16]:
#Creating new dataframes based on type of review 

AA_negative = AA_grouped.get_group("negative")
AA_negative=AA_negative.reset_index(drop=True)

AA_negative

In [17]:
#Creating new dataframes based on type of review 

AA_neutral = AA_grouped.get_group("neutral")
AA_neutral=AA_neutral.reset_index(drop=True)

AA_neutral

In [18]:
#creating separate dataframes for different airlines
UA_df = data_tweets_grouped.get_group("UA")
#Creating new dataframes based on type of review 

UA_grouped = UA_df.groupby("airline_sentiment")
UA_positive = UA_grouped.get_group("positive")
UA_positive = UA_positive.reset_index(drop=True)

UA_positive

,tweet_id,airline_sentiment,airline_sentiment_confidence,airline,name,retweet_count,text,tweet_date,tweet_time
0,567701830805618688,positive,1.0000,UA,scherzva,0,"@united New Apple crâpe, amazing! Live from UA...",2015-02-17,07:07
1,567729696662511616,positive,1.0000,UA,mchooyah,0,@united Thank you for the new Club at O'Hare. ...,2015-02-17,08:58
2,567733177590874114,positive,1.0000,UA,ClaudiaStClair,0,@united I appreciate the follow up.,2015-02-17,09:11
3,567733228690083841,positive,1.0000,UA,jsumiyasu,0,@united @jsumiyasu I am thankful to the Unite...,2015-02-17,09:12
4,567733609130233856,positive,0.7097,UA,BhutanOrient,0,@united no worries about the tweets. We all sh...,2015-02-17,09:13
...,...,...,...,...,...,...,...,...,...
487,570299819610251265,positive,1.0000,UA,BK_TheBri,0,@united Thanks. It is on the same ticket.,2015-02-24,11:10
488,570299889688702976,positive,0.6634,UA,nydia376,0,@united thanks,2015-02-24,11:11
489,570306733010264064,positive,0.3441,UA,rombaa,0,@united thanks -- we filled it out. How's our ...,2015-02-24,11:38
490,570307847281614848,positive,1.0000,UA,CoreyAStewart,0,@united Thanks for taking care of that MR!! Ha...,2015-02-24,11:42


In [19]:
#Creating new dataframes based on type of review 

UA_negative = UA_grouped.get_group("negative")
UA_negative = UA_negative.reset_index(drop=True)

UA_negative

In [20]:
#Creating new dataframes based on type of review 

UA_neutral = UA_grouped.get_group("neutral")
UA_neutral = UA_neutral.reset_index(drop=True)

UA_neutral

In [21]:
#creating separate dataframes for different airlines
US_df = data_tweets_grouped.get_group("US")
#Creating new dataframes based on type of review 

US_grouped = US_df.groupby("airline_sentiment")
US_positive = US_grouped.get_group("positive")
US_positive = US_positive.reset_index(drop=True)

US_positive

In [22]:
#Creating new dataframes based on type of review 

US_negative = US_grouped.get_group("negative")
US_negative = US_negative.reset_index(drop=True)

US_negative

In [23]:
#Creating new dataframes based on type of review 

US_neutral = US_grouped.get_group("neutral")
US_neutral = US_neutral.reset_index(drop=True)

US_neutral

In [24]:
#creating separate dataframes for different airlines
WN_df = data_tweets_grouped.get_group("WN")
#Creating new dataframes based on type of review 

WN_grouped = WN_df.groupby("airline_sentiment")
WN_positive = WN_grouped.get_group("positive")
WN_positive = WN_positive.reset_index(drop=True)

WN_positive

In [25]:
#Creating new dataframes based on type of review 

WN_negative = WN_grouped.get_group("negative")
WN_negative = WN_negative.reset_index(drop=True)

WN_negative

In [26]:
#Creating new dataframes based on type of review 

WN_neutral = WN_grouped.get_group("neutral")
WN_neutral = WN_neutral.reset_index(drop=True)

WN_neutral

In [27]:
#creating separate dataframes for different airlines
DL_df = data_tweets_grouped.get_group("DL")
#Creating new dataframes based on type of review 

DL_grouped = DL_df.groupby("airline_sentiment")
DL_positive = DL_grouped.get_group("positive")
DL_positive = DL_positive.reset_index(drop=True)

DL_positive

In [28]:
#Creating new dataframes based on type of review 

DL_negative = DL_grouped.get_group("negative")
DL_negative = DL_negative.reset_index(drop=True)

DL_negative

In [29]:
#Creating new dataframes based on type of review 

DL_neutral = DL_grouped.get_group("neutral")
DL_neutral = DL_neutral.reset_index(drop=True)

DL_neutral

In [30]:
#creating separate dataframes for different airlines
VX_df = data_tweets_grouped.get_group("VX")
#Creating new dataframes based on type of review 

VX_grouped = VX_df.groupby("airline_sentiment")
VX_positive = VX_grouped.get_group("positive")
VX_positive = VX_positive.reset_index(drop=True)

VX_positive

In [31]:
#Creating new dataframes based on type of review 

VX_negative = VX_grouped.get_group("negative")
VX_negative = VX_negative.reset_index(drop=True)

VX_negative

In [32]:
#Creating new dataframes based on type of review 

VX_neutral = VX_grouped.get_group("neutral")
VX_neutral = VX_neutral.reset_index(drop=True)

VX_neutral

## Data Loading (Matt)

In [33]:
#Import Database Key
from db_keys import db_key

#Connect to PostgreSQL Database
pg_engine = create_engine('postgresql://' + db_key + '@localhost:5432/test_1')

In [34]:
#Create Airlines Table
pg_engine.execute('CREATE TABLE "Airlines" ("IATA_CODE" VARCHAR(2) PRIMARY KEY, "AIRLINE" VARCHAR(50) NOT NULL);')

In [35]:
#Add Airlines Data Frame to Airlines Table

In [36]:
#Create Airports Table
pg_engine.execute('CREATE TABLE "Airports" ("IATA_CODE" VARCHAR(3) PRIMARY KEY, "AIRPORT" VARCHAR(50) NOT NULL, "CITY" VARCHAR(50) NOT NULL, "STATE" VARCHAR(2) NOT NULL, "COUNTRY" VARCHAR(3) NOT NULL);')

In [37]:
#Add Airports Data Frame to Airports Table

In [38]:
#Create UA Early Table
pg_engine.execute('CREATE TABLE "UA_Early" ("DAY_OF_WEEK" VARCHAR(10), "AIRLINE" VARCHAR(2), "FLIGHT_NUMBER" INT, "TAIL_NUMBER" VARCHAR(6), "ORIGIN_AIRPORT" VARCHAR(3), "DESTINATION_AIRPORT" VARCHAR(3), "SCHEDULED_DEPARTURE" VARCHAR(5), "DEPARTURE_TIME" VARCHAR(5), "DEPARTURE_DELAY" VARCHAR(5), "TAXI_OUT" VARCHAR(5), "WHEELS_OFF" VARCHAR(5), "SCHEDULED_TIME" VARCHAR(5), "ELAPSED_TIME" VARCHAR(5), "AIR_TIME" VARCHAR(5), "DISTANCE" INT, "WHEELS_ON" VARCHAR(5), "TAXI_IN" VARCHAR(5), "SCHEDULED_ARRIVAL" VARCHAR(5), "ARRIVAL_TIME" VARCHAR(5), "ARRIVAL_DELAY" VARCHAR(5), "TOTAL_DELAY" VARCHAR(5), "DATE" VARCHAR(10), FOREIGN KEY ("ORIGIN_AIRPORT") REFERENCES "Airports" ("IATA_CODE"), FOREIGN KEY ("DESTINATION_AIRPORT") REFERENCES "Airports" ("IATA_CODE"), FOREIGN KEY ("AIRLINE") REFERENCES "Airlines" ("IATA_CODE"));')

In [39]:
#Add UA Early Data Frame to UA Early Table

In [40]:
#Create AA Early Table
pg_engine.execute('CREATE TABLE "AA_Early" ("DAY_OF_WEEK" VARCHAR(10), "AIRLINE" VARCHAR(2), "FLIGHT_NUMBER" INT, "TAIL_NUMBER" VARCHAR(6), "ORIGIN_AIRPORT" VARCHAR(3), "DESTINATION_AIRPORT" VARCHAR(3), "SCHEDULED_DEPARTURE" VARCHAR(5), "DEPARTURE_TIME" VARCHAR(5), "DEPARTURE_DELAY" VARCHAR(5), "TAXI_OUT" VARCHAR(5), "WHEELS_OFF" VARCHAR(5), "SCHEDULED_TIME" VARCHAR(5), "ELAPSED_TIME" VARCHAR(5), "AIR_TIME" VARCHAR(5), "DISTANCE" INT, "WHEELS_ON" VARCHAR(5), "TAXI_IN" VARCHAR(5), "SCHEDULED_ARRIVAL" VARCHAR(5), "ARRIVAL_TIME" VARCHAR(5), "ARRIVAL_DELAY" VARCHAR(5), "TOTAL_DELAY" VARCHAR(5), "DATE" VARCHAR(10), FOREIGN KEY ("ORIGIN_AIRPORT") REFERENCES "Airports" ("IATA_CODE"), FOREIGN KEY ("DESTINATION_AIRPORT") REFERENCES "Airports" ("IATA_CODE"), FOREIGN KEY ("AIRLINE") REFERENCES "Airlines" ("IATA_CODE"));')

In [41]:
#Add AA Early Data Frame to AA Early Table

In [42]:
#Create DL Early Table
pg_engine.execute('CREATE TABLE "DL_Early" ("DAY_OF_WEEK" VARCHAR(10), "AIRLINE" VARCHAR(2), "FLIGHT_NUMBER" INT, "TAIL_NUMBER" VARCHAR(6), "ORIGIN_AIRPORT" VARCHAR(3), "DESTINATION_AIRPORT" VARCHAR(3), "SCHEDULED_DEPARTURE" VARCHAR(5), "DEPARTURE_TIME" VARCHAR(5), "DEPARTURE_DELAY" VARCHAR(5), "TAXI_OUT" VARCHAR(5), "WHEELS_OFF" VARCHAR(5), "SCHEDULED_TIME" VARCHAR(5), "ELAPSED_TIME" VARCHAR(5), "AIR_TIME" VARCHAR(5), "DISTANCE" INT, "WHEELS_ON" VARCHAR(5), "TAXI_IN" VARCHAR(5), "SCHEDULED_ARRIVAL" VARCHAR(5), "ARRIVAL_TIME" VARCHAR(5), "ARRIVAL_DELAY" VARCHAR(5), "TOTAL_DELAY" VARCHAR(5), "DATE" VARCHAR(10), FOREIGN KEY ("ORIGIN_AIRPORT") REFERENCES "Airports" ("IATA_CODE"), FOREIGN KEY ("DESTINATION_AIRPORT") REFERENCES "Airports" ("IATA_CODE"), FOREIGN KEY ("AIRLINE") REFERENCES "Airlines" ("IATA_CODE"));')

In [43]:
#Add DL Early Data Frame to DL Early Table

In [44]:
#Create US Early Table
pg_engine.execute('CREATE TABLE "US_Early" ("DAY_OF_WEEK" VARCHAR(10), "AIRLINE" VARCHAR(2), "FLIGHT_NUMBER" INT, "TAIL_NUMBER" VARCHAR(6), "ORIGIN_AIRPORT" VARCHAR(3), "DESTINATION_AIRPORT" VARCHAR(3), "SCHEDULED_DEPARTURE" VARCHAR(5), "DEPARTURE_TIME" VARCHAR(5), "DEPARTURE_DELAY" VARCHAR(5), "TAXI_OUT" VARCHAR(5), "WHEELS_OFF" VARCHAR(5), "SCHEDULED_TIME" VARCHAR(5), "ELAPSED_TIME" VARCHAR(5), "AIR_TIME" VARCHAR(5), "DISTANCE" INT, "WHEELS_ON" VARCHAR(5), "TAXI_IN" VARCHAR(5), "SCHEDULED_ARRIVAL" VARCHAR(5), "ARRIVAL_TIME" VARCHAR(5), "ARRIVAL_DELAY" VARCHAR(5), "TOTAL_DELAY" VARCHAR(5), "DATE" VARCHAR(10), FOREIGN KEY ("ORIGIN_AIRPORT") REFERENCES "Airports" ("IATA_CODE"), FOREIGN KEY ("DESTINATION_AIRPORT") REFERENCES "Airports" ("IATA_CODE"), FOREIGN KEY ("AIRLINE") REFERENCES "Airlines" ("IATA_CODE"));')

In [45]:
#Add US Early Data Frame to US Early Table

In [46]:
#Create WN Early Table
pg_engine.execute('CREATE TABLE "WN_Early" ("DAY_OF_WEEK" VARCHAR(10), "AIRLINE" VARCHAR(2), "FLIGHT_NUMBER" INT, "TAIL_NUMBER" VARCHAR(6), "ORIGIN_AIRPORT" VARCHAR(3), "DESTINATION_AIRPORT" VARCHAR(3), "SCHEDULED_DEPARTURE" VARCHAR(5), "DEPARTURE_TIME" VARCHAR(5), "DEPARTURE_DELAY" VARCHAR(5), "TAXI_OUT" VARCHAR(5), "WHEELS_OFF" VARCHAR(5), "SCHEDULED_TIME" VARCHAR(5), "ELAPSED_TIME" VARCHAR(5), "AIR_TIME" VARCHAR(5), "DISTANCE" INT, "WHEELS_ON" VARCHAR(5), "TAXI_IN" VARCHAR(5), "SCHEDULED_ARRIVAL" VARCHAR(5), "ARRIVAL_TIME" VARCHAR(5), "ARRIVAL_DELAY" VARCHAR(5), "TOTAL_DELAY" VARCHAR(5), "DATE" VARCHAR(10), FOREIGN KEY ("ORIGIN_AIRPORT") REFERENCES "Airports" ("IATA_CODE"), FOREIGN KEY ("DESTINATION_AIRPORT") REFERENCES "Airports" ("IATA_CODE"), FOREIGN KEY ("AIRLINE") REFERENCES "Airlines" ("IATA_CODE"));')

In [47]:
#Add WN Early Data Frame to WN Early Table

In [48]:
#Create VX Early Table
pg_engine.execute('CREATE TABLE "VX_Early" ("DAY_OF_WEEK" VARCHAR(10), "AIRLINE" VARCHAR(2), "FLIGHT_NUMBER" INT, "TAIL_NUMBER" VARCHAR(6), "ORIGIN_AIRPORT" VARCHAR(3), "DESTINATION_AIRPORT" VARCHAR(3), "SCHEDULED_DEPARTURE" VARCHAR(5), "DEPARTURE_TIME" VARCHAR(5), "DEPARTURE_DELAY" VARCHAR(5), "TAXI_OUT" VARCHAR(5), "WHEELS_OFF" VARCHAR(5), "SCHEDULED_TIME" VARCHAR(5), "ELAPSED_TIME" VARCHAR(5), "AIR_TIME" VARCHAR(5), "DISTANCE" INT, "WHEELS_ON" VARCHAR(5), "TAXI_IN" VARCHAR(5), "SCHEDULED_ARRIVAL" VARCHAR(5), "ARRIVAL_TIME" VARCHAR(5), "ARRIVAL_DELAY" VARCHAR(5), "TOTAL_DELAY" VARCHAR(5), "DATE" VARCHAR(10), FOREIGN KEY ("ORIGIN_AIRPORT") REFERENCES "Airports" ("IATA_CODE"), FOREIGN KEY ("DESTINATION_AIRPORT") REFERENCES "Airports" ("IATA_CODE"), FOREIGN KEY ("AIRLINE") REFERENCES "Airlines" ("IATA_CODE"));')

In [49]:
#Add VX Early Data Frame to VX Early Table

In [50]:
#Create UA Late Table
pg_engine.execute('CREATE TABLE "UA_Late" ("DAY_OF_WEEK" VARCHAR(10), "AIRLINE" VARCHAR(2), "FLIGHT_NUMBER" INT, "TAIL_NUMBER" VARCHAR(6), "ORIGIN_AIRPORT" VARCHAR(3), "DESTINATION_AIRPORT" VARCHAR(3), "SCHEDULED_DEPARTURE" VARCHAR(5), "DEPARTURE_TIME" VARCHAR(5), "DEPARTURE_DELAY" VARCHAR(5), "TAXI_OUT" VARCHAR(5), "WHEELS_OFF" VARCHAR(5), "SCHEDULED_TIME" VARCHAR(5), "ELAPSED_TIME" VARCHAR(5), "AIR_TIME" VARCHAR(5), "DISTANCE" INT, "WHEELS_ON" VARCHAR(5), "TAXI_IN" VARCHAR(5), "SCHEDULED_ARRIVAL" VARCHAR(5), "ARRIVAL_TIME" VARCHAR(5), "ARRIVAL_DELAY" VARCHAR(5), "TOTAL_DELAY" VARCHAR(5), "DATE" VARCHAR(10), FOREIGN KEY ("ORIGIN_AIRPORT") REFERENCES "Airports" ("IATA_CODE"), FOREIGN KEY ("DESTINATION_AIRPORT") REFERENCES "Airports" ("IATA_CODE"), FOREIGN KEY ("AIRLINE") REFERENCES "Airlines" ("IATA_CODE"));')

In [51]:
#Add UA Late Data Frame to UA Late Table

In [52]:
#Create AA Late Table
pg_engine.execute('CREATE TABLE "AA_Late" ("DAY_OF_WEEK" VARCHAR(10), "AIRLINE" VARCHAR(2), "FLIGHT_NUMBER" INT, "TAIL_NUMBER" VARCHAR(6), "ORIGIN_AIRPORT" VARCHAR(3), "DESTINATION_AIRPORT" VARCHAR(3), "SCHEDULED_DEPARTURE" VARCHAR(5), "DEPARTURE_TIME" VARCHAR(5), "DEPARTURE_DELAY" VARCHAR(5), "TAXI_OUT" VARCHAR(5), "WHEELS_OFF" VARCHAR(5), "SCHEDULED_TIME" VARCHAR(5), "ELAPSED_TIME" VARCHAR(5), "AIR_TIME" VARCHAR(5), "DISTANCE" INT, "WHEELS_ON" VARCHAR(5), "TAXI_IN" VARCHAR(5), "SCHEDULED_ARRIVAL" VARCHAR(5), "ARRIVAL_TIME" VARCHAR(5), "ARRIVAL_DELAY" VARCHAR(5), "TOTAL_DELAY" VARCHAR(5), "DATE" VARCHAR(10), FOREIGN KEY ("ORIGIN_AIRPORT") REFERENCES "Airports" ("IATA_CODE"), FOREIGN KEY ("DESTINATION_AIRPORT") REFERENCES "Airports" ("IATA_CODE"), FOREIGN KEY ("AIRLINE") REFERENCES "Airlines" ("IATA_CODE"));')

In [53]:
#Add AA Late Data Frame to AA Late Table

In [54]:
#Create DL Late Table
pg_engine.execute('CREATE TABLE "DL_Late" ("DAY_OF_WEEK" VARCHAR(10), "AIRLINE" VARCHAR(2), "FLIGHT_NUMBER" INT, "TAIL_NUMBER" VARCHAR(6), "ORIGIN_AIRPORT" VARCHAR(3), "DESTINATION_AIRPORT" VARCHAR(3), "SCHEDULED_DEPARTURE" VARCHAR(5), "DEPARTURE_TIME" VARCHAR(5), "DEPARTURE_DELAY" VARCHAR(5), "TAXI_OUT" VARCHAR(5), "WHEELS_OFF" VARCHAR(5), "SCHEDULED_TIME" VARCHAR(5), "ELAPSED_TIME" VARCHAR(5), "AIR_TIME" VARCHAR(5), "DISTANCE" INT, "WHEELS_ON" VARCHAR(5), "TAXI_IN" VARCHAR(5), "SCHEDULED_ARRIVAL" VARCHAR(5), "ARRIVAL_TIME" VARCHAR(5), "ARRIVAL_DELAY" VARCHAR(5), "TOTAL_DELAY" VARCHAR(5), "DATE" VARCHAR(10), FOREIGN KEY ("ORIGIN_AIRPORT") REFERENCES "Airports" ("IATA_CODE"), FOREIGN KEY ("DESTINATION_AIRPORT") REFERENCES "Airports" ("IATA_CODE"), FOREIGN KEY ("AIRLINE") REFERENCES "Airlines" ("IATA_CODE"));')

In [55]:
#Add DL Late Data Frame to DL Late Table

In [56]:
#Create US Late Table
pg_engine.execute('CREATE TABLE "US_Late" ("DAY_OF_WEEK" VARCHAR(10), "AIRLINE" VARCHAR(2), "FLIGHT_NUMBER" INT, "TAIL_NUMBER" VARCHAR(6), "ORIGIN_AIRPORT" VARCHAR(3), "DESTINATION_AIRPORT" VARCHAR(3), "SCHEDULED_DEPARTURE" VARCHAR(5), "DEPARTURE_TIME" VARCHAR(5), "DEPARTURE_DELAY" VARCHAR(5), "TAXI_OUT" VARCHAR(5), "WHEELS_OFF" VARCHAR(5), "SCHEDULED_TIME" VARCHAR(5), "ELAPSED_TIME" VARCHAR(5), "AIR_TIME" VARCHAR(5), "DISTANCE" INT, "WHEELS_ON" VARCHAR(5), "TAXI_IN" VARCHAR(5), "SCHEDULED_ARRIVAL" VARCHAR(5), "ARRIVAL_TIME" VARCHAR(5), "ARRIVAL_DELAY" VARCHAR(5), "TOTAL_DELAY" VARCHAR(5), "DATE" VARCHAR(10), FOREIGN KEY ("ORIGIN_AIRPORT") REFERENCES "Airports" ("IATA_CODE"), FOREIGN KEY ("DESTINATION_AIRPORT") REFERENCES "Airports" ("IATA_CODE"), FOREIGN KEY ("AIRLINE") REFERENCES "Airlines" ("IATA_CODE"));')

In [57]:
#Add US Late Data Frame to US Late Table

In [58]:
#Create WN Late Table
pg_engine.execute('CREATE TABLE "WN_Late" ("DAY_OF_WEEK" VARCHAR(10), "AIRLINE" VARCHAR(2), "FLIGHT_NUMBER" INT, "TAIL_NUMBER" VARCHAR(6), "ORIGIN_AIRPORT" VARCHAR(3), "DESTINATION_AIRPORT" VARCHAR(3), "SCHEDULED_DEPARTURE" VARCHAR(5), "DEPARTURE_TIME" VARCHAR(5), "DEPARTURE_DELAY" VARCHAR(5), "TAXI_OUT" VARCHAR(5), "WHEELS_OFF" VARCHAR(5), "SCHEDULED_TIME" VARCHAR(5), "ELAPSED_TIME" VARCHAR(5), "AIR_TIME" VARCHAR(5), "DISTANCE" INT, "WHEELS_ON" VARCHAR(5), "TAXI_IN" VARCHAR(5), "SCHEDULED_ARRIVAL" VARCHAR(5), "ARRIVAL_TIME" VARCHAR(5), "ARRIVAL_DELAY" VARCHAR(5), "TOTAL_DELAY" VARCHAR(5), "DATE" VARCHAR(10), FOREIGN KEY ("ORIGIN_AIRPORT") REFERENCES "Airports" ("IATA_CODE"), FOREIGN KEY ("DESTINATION_AIRPORT") REFERENCES "Airports" ("IATA_CODE"), FOREIGN KEY ("AIRLINE") REFERENCES "Airlines" ("IATA_CODE"));')

In [59]:
#Add WN Late Data Frame to WN Late Table

In [60]:
#Create VX Late Table
pg_engine.execute('CREATE TABLE "VX_Late" ("DAY_OF_WEEK" VARCHAR(10), "AIRLINE" VARCHAR(2), "FLIGHT_NUMBER" INT, "TAIL_NUMBER" VARCHAR(6), "ORIGIN_AIRPORT" VARCHAR(3), "DESTINATION_AIRPORT" VARCHAR(3), "SCHEDULED_DEPARTURE" VARCHAR(5), "DEPARTURE_TIME" VARCHAR(5), "DEPARTURE_DELAY" VARCHAR(5), "TAXI_OUT" VARCHAR(5), "WHEELS_OFF" VARCHAR(5), "SCHEDULED_TIME" VARCHAR(5), "ELAPSED_TIME" VARCHAR(5), "AIR_TIME" VARCHAR(5), "DISTANCE" INT, "WHEELS_ON" VARCHAR(5), "TAXI_IN" VARCHAR(5), "SCHEDULED_ARRIVAL" VARCHAR(5), "ARRIVAL_TIME" VARCHAR(5), "ARRIVAL_DELAY" VARCHAR(5), "TOTAL_DELAY" VARCHAR(5), "DATE" VARCHAR(10), FOREIGN KEY ("ORIGIN_AIRPORT") REFERENCES "Airports" ("IATA_CODE"), FOREIGN KEY ("DESTINATION_AIRPORT") REFERENCES "Airports" ("IATA_CODE"), FOREIGN KEY ("AIRLINE") REFERENCES "Airlines" ("IATA_CODE"));')

In [61]:
#Add VX Late Data Frame to VX Late Table

In [62]:
#Create UA Diverted Table
pg_engine.execute('CREATE TABLE "UA_Diverted" ("DAY_OF_WEEK" VARCHAR(10), "AIRLINE" VARCHAR(2), "FLIGHT_NUMBER" INT, "TAIL_NUMBER" VARCHAR(6), "ORIGIN_AIRPORT" VARCHAR(3), "DESTINATION_AIRPORT" VARCHAR(3), "SCHEDULED_DEPARTURE" VARCHAR(5), "DEPARTURE_TIME" VARCHAR(5), "DEPARTURE_DELAY" VARCHAR(5), "TAXI_OUT" VARCHAR(5), "WHEELS_OFF" VARCHAR(5), "SCHEDULED_TIME" VARCHAR(5), "DISTANCE" INT, "WHEELS_ON" VARCHAR(5), "TAXI_IN" VARCHAR(5), "SCHEDULED_ARRIVAL" VARCHAR(5), "ARRIVAL_TIME" VARCHAR(5), "DATE" VARCHAR(10), FOREIGN KEY ("ORIGIN_AIRPORT") REFERENCES "Airports" ("IATA_CODE"), FOREIGN KEY ("DESTINATION_AIRPORT") REFERENCES "Airports" ("IATA_CODE"), FOREIGN KEY ("AIRLINE") REFERENCES "Airlines" ("IATA_CODE"));')

In [63]:
#Add UA Diverted Data Frame to UA Diverted Table

In [64]:
#Create AA Diverted Table
pg_engine.execute('CREATE TABLE "AA_Diverted" ("DAY_OF_WEEK" VARCHAR(10), "AIRLINE" VARCHAR(2), "FLIGHT_NUMBER" INT, "TAIL_NUMBER" VARCHAR(6), "ORIGIN_AIRPORT" VARCHAR(3), "DESTINATION_AIRPORT" VARCHAR(3), "SCHEDULED_DEPARTURE" VARCHAR(5), "DEPARTURE_TIME" VARCHAR(5), "DEPARTURE_DELAY" VARCHAR(5), "TAXI_OUT" VARCHAR(5), "WHEELS_OFF" VARCHAR(5), "SCHEDULED_TIME" VARCHAR(5), "DISTANCE" INT, "WHEELS_ON" VARCHAR(5), "TAXI_IN" VARCHAR(5), "SCHEDULED_ARRIVAL" VARCHAR(5), "ARRIVAL_TIME" VARCHAR(5), "DATE" VARCHAR(10), FOREIGN KEY ("ORIGIN_AIRPORT") REFERENCES "Airports" ("IATA_CODE"), FOREIGN KEY ("DESTINATION_AIRPORT") REFERENCES "Airports" ("IATA_CODE"), FOREIGN KEY ("AIRLINE") REFERENCES "Airlines" ("IATA_CODE"));')

In [65]:
#Add AA Diverted Data Frame to AA Diverted Table

In [66]:
#Create DL Diverted Table
pg_engine.execute('CREATE TABLE "DL_Diverted" ("DAY_OF_WEEK" VARCHAR(10), "AIRLINE" VARCHAR(2), "FLIGHT_NUMBER" INT, "TAIL_NUMBER" VARCHAR(6), "ORIGIN_AIRPORT" VARCHAR(3), "DESTINATION_AIRPORT" VARCHAR(3), "SCHEDULED_DEPARTURE" VARCHAR(5), "DEPARTURE_TIME" VARCHAR(5), "DEPARTURE_DELAY" VARCHAR(5), "TAXI_OUT" VARCHAR(5), "WHEELS_OFF" VARCHAR(5), "SCHEDULED_TIME" VARCHAR(5), "DISTANCE" INT, "WHEELS_ON" VARCHAR(5), "TAXI_IN" VARCHAR(5), "SCHEDULED_ARRIVAL" VARCHAR(5), "ARRIVAL_TIME" VARCHAR(5), "DATE" VARCHAR(10), FOREIGN KEY ("ORIGIN_AIRPORT") REFERENCES "Airports" ("IATA_CODE"), FOREIGN KEY ("DESTINATION_AIRPORT") REFERENCES "Airports" ("IATA_CODE"), FOREIGN KEY ("AIRLINE") REFERENCES "Airlines" ("IATA_CODE"));')

In [67]:
#Add DL Diverted Data Frame to DL Diverted Table

In [68]:
#Create US Diverted Table
pg_engine.execute('CREATE TABLE "US_Diverted" ("DAY_OF_WEEK" VARCHAR(10), "AIRLINE" VARCHAR(2), "FLIGHT_NUMBER" INT, "TAIL_NUMBER" VARCHAR(6), "ORIGIN_AIRPORT" VARCHAR(3), "DESTINATION_AIRPORT" VARCHAR(3), "SCHEDULED_DEPARTURE" VARCHAR(5), "DEPARTURE_TIME" VARCHAR(5), "DEPARTURE_DELAY" VARCHAR(5), "TAXI_OUT" VARCHAR(5), "WHEELS_OFF" VARCHAR(5), "SCHEDULED_TIME" VARCHAR(5), "DISTANCE" INT, "WHEELS_ON" VARCHAR(5), "TAXI_IN" VARCHAR(5), "SCHEDULED_ARRIVAL" VARCHAR(5), "ARRIVAL_TIME" VARCHAR(5), "DATE" VARCHAR(10), FOREIGN KEY ("ORIGIN_AIRPORT") REFERENCES "Airports" ("IATA_CODE"), FOREIGN KEY ("DESTINATION_AIRPORT") REFERENCES "Airports" ("IATA_CODE"), FOREIGN KEY ("AIRLINE") REFERENCES "Airlines" ("IATA_CODE"));')

In [69]:
#Add US Diverted Data Frame to US Diverted Table

In [70]:
#Create WN Diverted Table
pg_engine.execute('CREATE TABLE "WN_Diverted" ("DAY_OF_WEEK" VARCHAR(10), "AIRLINE" VARCHAR(2), "FLIGHT_NUMBER" INT, "TAIL_NUMBER" VARCHAR(6), "ORIGIN_AIRPORT" VARCHAR(3), "DESTINATION_AIRPORT" VARCHAR(3), "SCHEDULED_DEPARTURE" VARCHAR(5), "DEPARTURE_TIME" VARCHAR(5), "DEPARTURE_DELAY" VARCHAR(5), "TAXI_OUT" VARCHAR(5), "WHEELS_OFF" VARCHAR(5), "SCHEDULED_TIME" VARCHAR(5), "DISTANCE" INT, "WHEELS_ON" VARCHAR(5), "TAXI_IN" VARCHAR(5), "SCHEDULED_ARRIVAL" VARCHAR(5), "ARRIVAL_TIME" VARCHAR(5), "DATE" VARCHAR(10), FOREIGN KEY ("ORIGIN_AIRPORT") REFERENCES "Airports" ("IATA_CODE"), FOREIGN KEY ("DESTINATION_AIRPORT") REFERENCES "Airports" ("IATA_CODE"), FOREIGN KEY ("AIRLINE") REFERENCES "Airlines" ("IATA_CODE"));')

In [71]:
#Add WN Diverted Data Frame to WN Diverted Table

In [72]:
#Create VX Diverted Table
pg_engine.execute('CREATE TABLE "VX_Diverted" ("DAY_OF_WEEK" VARCHAR(10), "AIRLINE" VARCHAR(2), "FLIGHT_NUMBER" INT, "TAIL_NUMBER" VARCHAR(6), "ORIGIN_AIRPORT" VARCHAR(3), "DESTINATION_AIRPORT" VARCHAR(3), "SCHEDULED_DEPARTURE" VARCHAR(5), "DEPARTURE_TIME" VARCHAR(5), "DEPARTURE_DELAY" VARCHAR(5), "TAXI_OUT" VARCHAR(5), "WHEELS_OFF" VARCHAR(5), "SCHEDULED_TIME" VARCHAR(5), "DISTANCE" INT, "WHEELS_ON" VARCHAR(5), "TAXI_IN" VARCHAR(5), "SCHEDULED_ARRIVAL" VARCHAR(5), "ARRIVAL_TIME" VARCHAR(5), "DATE" VARCHAR(10), FOREIGN KEY ("ORIGIN_AIRPORT") REFERENCES "Airports" ("IATA_CODE"), FOREIGN KEY ("DESTINATION_AIRPORT") REFERENCES "Airports" ("IATA_CODE"), FOREIGN KEY ("AIRLINE") REFERENCES "Airlines" ("IATA_CODE"));')

In [73]:
#Add VX Diverted Data Frame to VX Diverted Table

In [74]:
#Create UA Cancelled Table
pg_engine.execute('CREATE TABLE "UA_Cancelled" ("DAY_OF_WEEK" VARCHAR(10), "AIRLINE" VARCHAR(2), "FLIGHT_NUMBER" INT, "TAIL_NUMBER" VARCHAR(6), "ORIGIN_AIRPORT" VARCHAR(3), "DESTINATION_AIRPORT" VARCHAR(3), "SCHEDULED_DEPARTURE" VARCHAR(5), "SCHEDULED_TIME" VARCHAR(5), "DISTANCE" INT, "SCHEDULED_ARRIVAL" VARCHAR(5), "DATE" VARCHAR(10), FOREIGN KEY ("ORIGIN_AIRPORT") REFERENCES "Airports" ("IATA_CODE"), FOREIGN KEY ("DESTINATION_AIRPORT") REFERENCES "Airports" ("IATA_CODE"), FOREIGN KEY ("AIRLINE") REFERENCES "Airlines" ("IATA_CODE"));')

In [75]:
#Add UA Cancelled Data Frame to UA Cancelled Table

In [76]:
#Create AA Cancelled Table
pg_engine.execute('CREATE TABLE "AA_Cancelled" ("DAY_OF_WEEK" VARCHAR(10), "AIRLINE" VARCHAR(2), "FLIGHT_NUMBER" INT, "TAIL_NUMBER" VARCHAR(6), "ORIGIN_AIRPORT" VARCHAR(3), "DESTINATION_AIRPORT" VARCHAR(3), "SCHEDULED_DEPARTURE" VARCHAR(5), "SCHEDULED_TIME" VARCHAR(5), "DISTANCE" INT, "SCHEDULED_ARRIVAL" VARCHAR(5), "DATE" VARCHAR(10), FOREIGN KEY ("ORIGIN_AIRPORT") REFERENCES "Airports" ("IATA_CODE"), FOREIGN KEY ("DESTINATION_AIRPORT") REFERENCES "Airports" ("IATA_CODE"), FOREIGN KEY ("AIRLINE") REFERENCES "Airlines" ("IATA_CODE"));')

In [77]:
#Add AA Cancelled Data Frame to AA Cancelled Table

In [78]:
#Create DL Cancelled Table
pg_engine.execute('CREATE TABLE "DL_Cancelled" ("DAY_OF_WEEK" VARCHAR(10), "AIRLINE" VARCHAR(2), "FLIGHT_NUMBER" INT, "TAIL_NUMBER" VARCHAR(6), "ORIGIN_AIRPORT" VARCHAR(3), "DESTINATION_AIRPORT" VARCHAR(3), "SCHEDULED_DEPARTURE" VARCHAR(5), "SCHEDULED_TIME" VARCHAR(5), "DISTANCE" INT, "SCHEDULED_ARRIVAL" VARCHAR(5), "DATE" VARCHAR(10), FOREIGN KEY ("ORIGIN_AIRPORT") REFERENCES "Airports" ("IATA_CODE"), FOREIGN KEY ("DESTINATION_AIRPORT") REFERENCES "Airports" ("IATA_CODE"), FOREIGN KEY ("AIRLINE") REFERENCES "Airlines" ("IATA_CODE"));')

In [79]:
#Add DL Cancelled Data Frame to DL Cancelled Table

In [80]:
#Create US Cancelled Table
pg_engine.execute('CREATE TABLE "US_Cancelled" ("DAY_OF_WEEK" VARCHAR(10), "AIRLINE" VARCHAR(2), "FLIGHT_NUMBER" INT, "TAIL_NUMBER" VARCHAR(6), "ORIGIN_AIRPORT" VARCHAR(3), "DESTINATION_AIRPORT" VARCHAR(3), "SCHEDULED_DEPARTURE" VARCHAR(5), "SCHEDULED_TIME" VARCHAR(5), "DISTANCE" INT, "SCHEDULED_ARRIVAL" VARCHAR(5), "DATE" VARCHAR(10), FOREIGN KEY ("ORIGIN_AIRPORT") REFERENCES "Airports" ("IATA_CODE"), FOREIGN KEY ("DESTINATION_AIRPORT") REFERENCES "Airports" ("IATA_CODE"), FOREIGN KEY ("AIRLINE") REFERENCES "Airlines" ("IATA_CODE"));')

In [81]:
#Add US Cancelled Data Frame to US Cancelled Table

In [82]:
#Create WN Cancelled Table
pg_engine.execute('CREATE TABLE "WN_Cancelled" ("DAY_OF_WEEK" VARCHAR(10), "AIRLINE" VARCHAR(2), "FLIGHT_NUMBER" INT, "TAIL_NUMBER" VARCHAR(6), "ORIGIN_AIRPORT" VARCHAR(3), "DESTINATION_AIRPORT" VARCHAR(3), "SCHEDULED_DEPARTURE" VARCHAR(5), "SCHEDULED_TIME" VARCHAR(5), "DISTANCE" INT, "SCHEDULED_ARRIVAL" VARCHAR(5), "DATE" VARCHAR(10), FOREIGN KEY ("ORIGIN_AIRPORT") REFERENCES "Airports" ("IATA_CODE"), FOREIGN KEY ("DESTINATION_AIRPORT") REFERENCES "Airports" ("IATA_CODE"), FOREIGN KEY ("AIRLINE") REFERENCES "Airlines" ("IATA_CODE"));')

In [83]:
#Add WN Cancelled Data Frame to WN Cancelled Table

In [84]:
#Create VX Cancelled Table
pg_engine.execute('CREATE TABLE "VX_Cancelled" ("DAY_OF_WEEK" VARCHAR(10), "AIRLINE" VARCHAR(2), "FLIGHT_NUMBER" INT, "TAIL_NUMBER" VARCHAR(6), "ORIGIN_AIRPORT" VARCHAR(3), "DESTINATION_AIRPORT" VARCHAR(3), "SCHEDULED_DEPARTURE" VARCHAR(5), "SCHEDULED_TIME" VARCHAR(5), "DISTANCE" INT, "SCHEDULED_ARRIVAL" VARCHAR(5), "DATE" VARCHAR(10), FOREIGN KEY ("ORIGIN_AIRPORT") REFERENCES "Airports" ("IATA_CODE"), FOREIGN KEY ("DESTINATION_AIRPORT") REFERENCES "Airports" ("IATA_CODE"), FOREIGN KEY ("AIRLINE") REFERENCES "Airlines" ("IATA_CODE"));')

In [85]:
#Add VX Cancelled Data Frame to VX Cancelled Table

In [89]:
#Create UA Positive Table
pg_engine.execute('CREATE TABLE "UA_Positive" ("tweet_id" VARCHAR(100) PRIMARY KEY, "airline_sentiment" VARCHAR(10), "airline_sentiment_confidence" FLOAT, "airline" VARCHAR(2), "name" VARCHAR(50), "retweet_count" INT, "text" VARCHAR(300), "tweet_date" VARCHAR(10), "tweet_time" VARCHAR(5), FOREIGN KEY ("airline") REFERENCES "Airlines" ("IATA_CODE"));')

In [1]:
#Add UA Positive Data Frame to UA Positive Table

In [ ]:
#Create AA Positive Table
pg_engine.execute('CREATE TABLE "AA_Positive" ("tweet_id" VARCHAR(100) PRIMARY KEY, "airline_sentiment" VARCHAR(10), "airline_sentiment_confidence" FLOAT, "airline" VARCHAR(2), "name" VARCHAR(50), "retweet_count" INT, "text" VARCHAR(300), "tweet_date" VARCHAR(10), "tweet_time" VARCHAR(5), FOREIGN KEY ("airline") REFERENCES "Airlines" ("IATA_CODE"));')

In [ ]:
#Add AA Positive Data Frame to AA Positive Table

In [ ]:
#Create DL Positive Table
pg_engine.execute('CREATE TABLE "DL_Positive" ("tweet_id" VARCHAR(100) PRIMARY KEY, "airline_sentiment" VARCHAR(10), "airline_sentiment_confidence" FLOAT, "airline" VARCHAR(2), "name" VARCHAR(50), "retweet_count" INT, "text" VARCHAR(300), "tweet_date" VARCHAR(10), "tweet_time" VARCHAR(5), FOREIGN KEY ("airline") REFERENCES "Airlines" ("IATA_CODE"));')

In [ ]:
#Add DL Positive Data Frame to DL Positive Table

In [ ]:
#Create US Positive Table
pg_engine.execute('CREATE TABLE "US_Positive" ("tweet_id" VARCHAR(100) PRIMARY KEY, "airline_sentiment" VARCHAR(10), "airline_sentiment_confidence" FLOAT, "airline" VARCHAR(2), "name" VARCHAR(50), "retweet_count" INT, "text" VARCHAR(300), "tweet_date" VARCHAR(10), "tweet_time" VARCHAR(5), FOREIGN KEY ("airline") REFERENCES "Airlines" ("IATA_CODE"));')

In [ ]:
#Add US Positive Data Frame to US Positive Table

In [ ]:
#Create WN Positive Table
pg_engine.execute('CREATE TABLE "WN_Positive" ("tweet_id" VARCHAR(100) PRIMARY KEY, "airline_sentiment" VARCHAR(10), "airline_sentiment_confidence" FLOAT, "airline" VARCHAR(2), "name" VARCHAR(50), "retweet_count" INT, "text" VARCHAR(300), "tweet_date" VARCHAR(10), "tweet_time" VARCHAR(5), FOREIGN KEY ("airline") REFERENCES "Airlines" ("IATA_CODE"));')

In [ ]:
#Add WN Positive Data Frame to WN Positive Table

In [ ]:
#Create VX Positive Table
pg_engine.execute('CREATE TABLE "VX_Positive" ("tweet_id" VARCHAR(100) PRIMARY KEY, "airline_sentiment" VARCHAR(10), "airline_sentiment_confidence" FLOAT, "airline" VARCHAR(2), "name" VARCHAR(50), "retweet_count" INT, "text" VARCHAR(300), "tweet_date" VARCHAR(10), "tweet_time" VARCHAR(5), FOREIGN KEY ("airline") REFERENCES "Airlines" ("IATA_CODE"));')

In [ ]:
#Add VX Positive Data Frame to VX Positive Table

In [ ]:
#Create UA Neutral Table
pg_engine.execute('CREATE TABLE "UA_Neutral" ("tweet_id" VARCHAR(100) PRIMARY KEY, "airline_sentiment" VARCHAR(10), "airline_sentiment_confidence" FLOAT, "airline" VARCHAR(2), "name" VARCHAR(50), "retweet_count" INT, "text" VARCHAR(300), "tweet_date" VARCHAR(10), "tweet_time" VARCHAR(5), FOREIGN KEY ("airline") REFERENCES "Airlines" ("IATA_CODE"));')

In [ ]:
#Add UA Neutral Data Frame to UA Neutral Table

In [ ]:
#Create AA Neutral Table
pg_engine.execute('CREATE TABLE "AA_Neutral" ("tweet_id" VARCHAR(100) PRIMARY KEY, "airline_sentiment" VARCHAR(10), "airline_sentiment_confidence" FLOAT, "airline" VARCHAR(2), "name" VARCHAR(50), "retweet_count" INT, "text" VARCHAR(300), "tweet_date" VARCHAR(10), "tweet_time" VARCHAR(5), FOREIGN KEY ("airline") REFERENCES "Airlines" ("IATA_CODE"));')

In [ ]:
#Add AA Neutral Data Frame to AA Neutral Table

In [ ]:
#Create DL Neutral Table
pg_engine.execute('CREATE TABLE "DL_Neutral" ("tweet_id" VARCHAR(100) PRIMARY KEY, "airline_sentiment" VARCHAR(10), "airline_sentiment_confidence" FLOAT, "airline" VARCHAR(2), "name" VARCHAR(50), "retweet_count" INT, "text" VARCHAR(300), "tweet_date" VARCHAR(10), "tweet_time" VARCHAR(5), FOREIGN KEY ("airline") REFERENCES "Airlines" ("IATA_CODE"));')

In [ ]:
#Add DL Neutral Data Frame to DL Neutral Table

In [ ]:
#Create US Neutral Table
pg_engine.execute('CREATE TABLE "US_Neutral" ("tweet_id" VARCHAR(100) PRIMARY KEY, "airline_sentiment" VARCHAR(10), "airline_sentiment_confidence" FLOAT, "airline" VARCHAR(2), "name" VARCHAR(50), "retweet_count" INT, "text" VARCHAR(300), "tweet_date" VARCHAR(10), "tweet_time" VARCHAR(5), FOREIGN KEY ("airline") REFERENCES "Airlines" ("IATA_CODE"));')

In [ ]:
#Add US Neutral Data Frame to US Neutral Table

In [ ]:
#Create WN Neutral Table
pg_engine.execute('CREATE TABLE "WN_Neutral" ("tweet_id" VARCHAR(100) PRIMARY KEY, "airline_sentiment" VARCHAR(10), "airline_sentiment_confidence" FLOAT, "airline" VARCHAR(2), "name" VARCHAR(50), "retweet_count" INT, "text" VARCHAR(300), "tweet_date" VARCHAR(10), "tweet_time" VARCHAR(5), FOREIGN KEY ("airline") REFERENCES "Airlines" ("IATA_CODE"));')

In [ ]:
#Add WN Neutral Data Frame to WN Neutral Table

In [ ]:
#Create VX Neutral Table
pg_engine.execute('CREATE TABLE "VX_Neutral" ("tweet_id" VARCHAR(100) PRIMARY KEY, "airline_sentiment" VARCHAR(10), "airline_sentiment_confidence" FLOAT, "airline" VARCHAR(2), "name" VARCHAR(50), "retweet_count" INT, "text" VARCHAR(300), "tweet_date" VARCHAR(10), "tweet_time" VARCHAR(5), FOREIGN KEY ("airline") REFERENCES "Airlines" ("IATA_CODE"));')

In [ ]:
#Add VX Neutral Data Frame to VX Neutral Table

In [ ]:
#Create UA Negative Table
pg_engine.execute('CREATE TABLE "UA_Negative" ("tweet_id" VARCHAR(100) PRIMARY KEY, "airline_sentiment" VARCHAR(10), "airline_sentiment_confidence" FLOAT, "airline" VARCHAR(2), "name" VARCHAR(50), "retweet_count" INT, "text" VARCHAR(300), "tweet_date" VARCHAR(10), "tweet_time" VARCHAR(5), FOREIGN KEY ("airline") REFERENCES "Airlines" ("IATA_CODE"));')

In [ ]:
#Add UA Negative Data Frame to UA Negative Table

In [ ]:
#Create AA Negative Table
pg_engine.execute('CREATE TABLE "AA_Negative" ("tweet_id" VARCHAR(100) PRIMARY KEY, "airline_sentiment" VARCHAR(10), "airline_sentiment_confidence" FLOAT, "airline" VARCHAR(2), "name" VARCHAR(50), "retweet_count" INT, "text" VARCHAR(300), "tweet_date" VARCHAR(10), "tweet_time" VARCHAR(5), FOREIGN KEY ("airline") REFERENCES "Airlines" ("IATA_CODE"));')

In [ ]:
#Add AA Neutral Data Frame to AA Neutral Table

In [ ]:
#Create DL Negative Table
pg_engine.execute('CREATE TABLE "DL_Negative" ("tweet_id" VARCHAR(100) PRIMARY KEY, "airline_sentiment" VARCHAR(10), "airline_sentiment_confidence" FLOAT, "airline" VARCHAR(2), "name" VARCHAR(50), "retweet_count" INT, "text" VARCHAR(300), "tweet_date" VARCHAR(10), "tweet_time" VARCHAR(5), FOREIGN KEY ("airline") REFERENCES "Airlines" ("IATA_CODE"));')

In [ ]:
#Add DL Neutral Data Frame to DL Neutral Table

In [ ]:
#Create US Negative Table
pg_engine.execute('CREATE TABLE "US_Negative" ("tweet_id" VARCHAR(100) PRIMARY KEY, "airline_sentiment" VARCHAR(10), "airline_sentiment_confidence" FLOAT, "airline" VARCHAR(2), "name" VARCHAR(50), "retweet_count" INT, "text" VARCHAR(300), "tweet_date" VARCHAR(10), "tweet_time" VARCHAR(5), FOREIGN KEY ("airline") REFERENCES "Airlines" ("IATA_CODE"));')

In [ ]:
#Add US Neutral Data Frame to US Neutral Table

In [ ]:
#Create WN Negative Table
pg_engine.execute('CREATE TABLE "WN_Negative" ("tweet_id" VARCHAR(100) PRIMARY KEY, "airline_sentiment" VARCHAR(10), "airline_sentiment_confidence" FLOAT, "airline" VARCHAR(2), "name" VARCHAR(50), "retweet_count" INT, "text" VARCHAR(300), "tweet_date" VARCHAR(10), "tweet_time" VARCHAR(5), FOREIGN KEY ("airline") REFERENCES "Airlines" ("IATA_CODE"));')

In [ ]:
#Add WN Neutral Data Frame to WN Neutral Table

In [ ]:
#Create VX Negative Table
pg_engine.execute('CREATE TABLE "VX_Negative" ("tweet_id" VARCHAR(100) PRIMARY KEY, "airline_sentiment" VARCHAR(10), "airline_sentiment_confidence" FLOAT, "airline" VARCHAR(2), "name" VARCHAR(50), "retweet_count" INT, "text" VARCHAR(300), "tweet_date" VARCHAR(10), "tweet_time" VARCHAR(5), FOREIGN KEY ("airline") REFERENCES "Airlines" ("IATA_CODE"));')

In [ ]:
#Add VX Neutral Data Frame to VX Neutral Table

# Project Analysis (Matt)

The purpose of this project was to perform an Extract-Transform-Load (ETL) process on Feruary 2015 airline flight performance and customer tweets, based on datasets obtained from Kaggle. The SQLAlchemy and Pandas modules in Python were used to import, clean, and process the data, while the former was used to output the transformed data into an SQL database. Discussed below are the specific actions taken to perform each step of the ETL process.

## Data Extraction

The extraction phase of the ETL process included importing and inspecting the data, with the latter including the identification all issues and inconsistencies with the source data. To begin, there were four files that needed to be imported, which included three CSV files and one SQLite database file. A Pandas data frame was created for each of the former, which contained flight, airline, and airport information respectively. Before importing the latter, a connection was established through the SQLite engine to the database file and the table name was exported. The tweet information table was then extracted from the database directly to a Pandas data frame for inspection.

With the import process complete, the next step was to thoroughly inspect each of the data frames. Upon the completion of this process, several issues and inconsistencies were found. First, the date range of the customer tweet data was narrower than the date range of the flight performance data. Second, several columns in the flight information and tweet information data frames were missing large amounts of data. Third, the date and time values were not consistent between the flight and tweet information data frames. Fourth, the tweet information data frame contained fewer airlines than the flight information data frame. Finally, the latitude and longitude information for certain airports was incomplete in the airport information data frame.

## Data Transformation

Having established the issues with the imported data, the first step in the transformation phase was to clean each of the data frames. This was a multi-step process for each data frame, starting with the flight information data frame. TBD

## Data Loading

TBD